In [68]:
import pandas as pd
import numpy as np
import tensorflow as TF
from typing import Tuple, List
import cv2
import os
from sklearn.model_selection import train_test_split
import keras
CATEGORIES = [0, 1, 2, 3, 4, 5]
EPOCHS = 50
TEST = 0.3

In [69]:
def load_signs_dataset(dataDir : str): # type: ignore
    images = []
    labels = []
    for SingleCategory in CATEGORIES:
        category = os.path.join(dataDir, str(SingleCategory))
        
        for image in os.listdir(category):
            img_path = os.path.join(category, image)
            finalImage = cv2.imread(img_path)
            
            if finalImage is not None:
                img = cv2.resize(finalImage, (64, 64))
                images.append(img)
                
                labels.append(SingleCategory)
    
    return (np.array(images), np.array(labels))
                

In [70]:
images, labels = load_signs_dataset("DataSet/archive")

X_train_orig, X_test_orig, Y_train_orig, Y_test_orig= train_test_split(
    np.array(images), np.array(labels), test_size=TEST
)

print(X_train_orig.shape)
print(Y_train_orig.shape)
print(X_test_orig.shape)
print(Y_test_orig.shape)

(865, 64, 64, 3)
(865,)
(372, 64, 64, 3)
(372,)


In [71]:
def model(input_shape):
    input_img = TF.keras.Input(shape=input_shape)
    Z1 = TF.keras.layers.Conv2D(filters=8, kernel_size=(4,4), padding="same")(input_img)
    
    A1 = TF.keras.layers.ReLU()(Z1)
    
    P1 = TF.keras.layers.MaxPool2D(pool_size=(8,8), strides=8, padding="same")(A1)
    
    Z2 = TF.keras.layers.Conv2D(filters=16, kernel_size=(2,2), strides=1, padding="same")(P1)
    
    A2 = TF.keras.layers.ReLU()(Z2)
    
    P2 = TF.keras.layers.MaxPool2D(pool_size=(4,4), strides=4, padding="same")(A2)
    
    
    F = TF.keras.layers.Flatten()(P2)
    
    outpus = TF.keras.layers.Dense(units= 6, activation="softmax")(F)
    
    model = TF.keras.Model(input_img,outpus)
    return model

In [ ]:
ConvModel = model((64,64,3))
ConvModel.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
    )

Y_train_oh = TF.keras.utils.to_categorical(Y_train_orig, num_classes=6)
Y_test_oh = TF.keras.utils.to_categorical(Y_test_orig, num_classes=6)
ConvModel.fit(X_train_orig/255.0, Y_train_oh, epochs=EPOCHS, validation_data=(X_test_orig/255.0, Y_test_oh))
ConvModel.summary()


Epoch 1/50
28/28 [==============================] - 2s 37ms/step - loss: 1.7975 - accuracy: 0.1561 - val_loss: 1.7932 - val_accuracy: 0.1774
Epoch 2/50
28/28 [==============================] - 1s 37ms/step - loss: 1.7906 - accuracy: 0.1861 - val_loss: 1.7939 - val_accuracy: 0.1774
Epoch 3/50
28/28 [==============================] - 1s 43ms/step - loss: 1.7831 - accuracy: 0.2243 - val_loss: 1.7826 - val_accuracy: 0.2581
Epoch 4/50
28/28 [==============================] - 1s 42ms/step - loss: 1.7730 - accuracy: 0.2393 - val_loss: 1.7760 - val_accuracy: 0.1747
Epoch 5/50
28/28 [==============================] - 1s 40ms/step - loss: 1.7592 - accuracy: 0.2694 - val_loss: 1.7772 - val_accuracy: 0.1344
Epoch 6/50
28/28 [==============================] - 1s 42ms/step - loss: 1.7412 - accuracy: 0.3202 - val_loss: 1.7478 - val_accuracy: 0.2688
Epoch 7/50
28/28 [==============================] - 1s 41ms/step - loss: 1.7188 - accuracy: 0.3283 - val_loss: 1.7517 - val_accuracy: 0.1344
Epoch 8/50
28

In [ ]:
test_loss, img_accuracy = ConvModel.evaluate(X_test_orig, Y_test_oh)

if img_accuracy >= 0.85:
    ConvModel.save("HandDetector.h5")

12/12 [==============================] - 0s 9ms/step - loss: 28.9139 - accuracy: 0.8737
